In [1]:
from pathlib import Path
from dotenv import load_dotenv
import openai

from rag_diary.config import Config

current_folder = Path(globals()['_dh'][0])
load_dotenv(Path(current_folder).parent / ".env")

config = Config()
openai.api_key = config.OPENAI_API_KEY

In [2]:
from rag_diary.vector_store_chromadb import VectorStoreChromadb, get_chromadb_collection
from langchain_openai.embeddings import OpenAIEmbeddings
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

db_collection_name = "organ_trail_spd"
db_path = Path(current_folder).joinpath("../db/private.db.chroma") 
embedding_function = OpenAIEmbeddings(api_key=config.OPENAI_API_KEY, model=config.OPENAI_MODEL_NAME)

collection_name = "langchain_retrival"
embeddings = OpenAIEmbeddings()
client = chromadb.PersistentClient(path=str(db_path.absolute()))
collection = get_chromadb_collection(
    client,
    collection_name=db_collection_name,
    embedding_function=embedding_function
)
vector_store = VectorStoreChromadb(client=client, collection=collection)

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
db = Chroma(client=vector_store.client, collection_name=collection.name, embedding_function=OpenAIEmbeddings())

In [4]:
# Prepare nltk token resources

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

2024-02-04 22:35:32.657 Python[14269:784634] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [29]:
from langchain_community.document_loaders import UnstructuredHTMLLoader
# Diary source https://www.gutenberg.org/ebooks/4200
document_path = db_path = Path(current_folder).joinpath("../local_storage/samuel_pepys_diary.html") 

loader = UnstructuredHTMLLoader(str(document_path.absolute()))
pages = loader.load_and_split()

In [30]:
print(len(pages))
print(pages[0])

2428
page_content='The Project Gutenberg eBook of The Diary of Samuel Pepys — Complete\n\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat \n\nwww.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\n\nTitle: The Diary of Samuel Pepys — Complete\n\nAuthor: Samuel Pepys\n\nEditor: Baron Richard Griffin Braybrooke\n\nHenry B. Wheatley\n\nTranslator: Mynors Bright\n\nRelease date: October 31, 2004 [eBook #4200]\n\nMost recently updated: January 27, 2021\n\nLanguage: English\n\nCredits: Produced by David Widger\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE DIARY OF SAMUEL PEPYS — COMPLETE ***\n\nTHE DIARY OF SAMUEL PEPYS M.A. F.R.S.\n\nCL

In [31]:
db.add_documents(documents=pages)

['e304621c-c390-11ee-ad2a-acde48001122',
 'e3046460-c390-11ee-ad2a-acde48001122',
 'e30464c4-c390-11ee-ad2a-acde48001122',
 'e3046500-c390-11ee-ad2a-acde48001122',
 'e304653c-c390-11ee-ad2a-acde48001122',
 'e304656e-c390-11ee-ad2a-acde48001122',
 'e30465aa-c390-11ee-ad2a-acde48001122',
 'e30465e6-c390-11ee-ad2a-acde48001122',
 'e3046618-c390-11ee-ad2a-acde48001122',
 'e3046654-c390-11ee-ad2a-acde48001122',
 'e3046686-c390-11ee-ad2a-acde48001122',
 'e30466b8-c390-11ee-ad2a-acde48001122',
 'e30466f4-c390-11ee-ad2a-acde48001122',
 'e3046726-c390-11ee-ad2a-acde48001122',
 'e3046758-c390-11ee-ad2a-acde48001122',
 'e3046794-c390-11ee-ad2a-acde48001122',
 'e30467c6-c390-11ee-ad2a-acde48001122',
 'e3046802-c390-11ee-ad2a-acde48001122',
 'e304683e-c390-11ee-ad2a-acde48001122',
 'e3046870-c390-11ee-ad2a-acde48001122',
 'e30468ac-c390-11ee-ad2a-acde48001122',
 'e30468de-c390-11ee-ad2a-acde48001122',
 'e304691a-c390-11ee-ad2a-acde48001122',
 'e304694c-c390-11ee-ad2a-acde48001122',
 'e3046988-c390-

In [5]:
retriever = db.as_retriever()
query = "What happened during the fire"

docs = retriever.get_relevant_documents(query)
print(*[doc.page_content for doc in docs])

2nd (Lord’s day). Some of our mayds sitting up late last night to get
      things ready against our feast to-day, Jane called us up about three in
      the morning, to tell us of a great fire they saw in the City. So I rose
      and slipped on my nightgowne, and went to her window, and thought it to be
      on the backside of Marke-lane at the farthest; but, being unused to such
      fires as followed, I thought it far enough off; and so went to bed again
      and to sleep. About seven rose again to dress myself, and there looked out
      at the window, and saw the fire not so much as it was and further off. So
      to my closett to set things to rights after yesterday’s cleaning. By and
      by Jane comes and tells me that she hears that above 300 houses have been
      burned down to-night by the fire we saw, and that it is now burning down
      all Fish-street, by London Bridge. So I made myself ready presently, and
      walked to the Tower, and there got up upon one of t

In [6]:
for doc in docs:
    print(doc.metadata)
    

{'source': '/Users/tonail_/code/rag-diary/notebooks/../local_storage/samuel_pepys_diary.html'}
{'source': '/Users/tonail_/code/rag-diary/notebooks/../local_storage/samuel_pepys_diary.html'}
{'source': '/Users/tonail_/code/rag-diary/notebooks/../local_storage/samuel_pepys_diary.html'}
{'source': '/Users/tonail_/code/rag-diary/notebooks/../local_storage/samuel_pepys_diary.html'}


In [7]:
from langchain_core.documents import Document
from langchain_community.document_transformers import LongContextReorder

def rerank_docs(docs: list[Document]):
    # Reorder the documents:
    # Less relevant document will be at the middle of the list and more
    # relevant elements at beginning / end.
    reordering = LongContextReorder()
    return reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs = rerank_docs(docs)


In [8]:
from langchain.prompts import PromptTemplate

template = """You are a helpful assistant who identifies as a personal diary.
You should respond in a nurturing tone. 
You are a medical professional with a specialization in mental health. 
You should not make medical diagnoses.
When it makes sense, you should reinforce positive actions.

Given this text extracts:
# -----
# {context}
# -----
# Please answer the following question for the author of the extracts.
# The user is the writer of the extracts, answer as if you are speaking to the writer. 
# The answer should Reflect on the users life.
# The answer should be factual and not embellish:
# bellow your response provide a few sources from the extracts.
# sources should contain `document_id` and no more than the first 240 characters of `text` 
# {query}"""
document_variable_name = "context"
prompt = PromptTemplate(
    template=template,
    input_variables=["query", document_variable_name ],
)

  

In [9]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_openai import OpenAI
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)

llm = OpenAI()
# The prompt here should take as an input variable the
# `document_variable_name`

llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context"
)


In [10]:
def query_agent(query: str):
    documents = db.max_marginal_relevance_search(query)
    print(len(documents))
    reranked_docs = rerank_docs(documents)
    
    return chain.invoke( query=query, input={"query": query, "input_documents": reranked_docs})
    

In [11]:
res = query_agent(query = "What happened during the fire")
res["output_text"]

4


' in London in 1666?\n\n\nThe fire in London in 1666, also known as the Great Fire of London, was a major disaster that destroyed a large part of the city. It started in the early morning of September 2nd, 1666, in the King\'s baker\'s house in Pudding Lane. Due to the dry weather and wooden buildings, the fire spread quickly and burned for four days. It destroyed over 13,000 houses and 87 churches, including St. Paul\'s Cathedral. The estimated death toll was only a few, but many people lost their homes and livelihoods. It also had a significant impact on the city\'s economy and led to major rebuilding efforts. (document_id: 1, text: "but the wind and tide being against us," - "not only these, but their warehouses at their Hall," - "and eat and were merry," - "and then with my Lady Pen and Pegg" - "all the great booksellers almost undone") (document_id: 2, text: "and eat and were merry," - "and then with my Lady Pen and Pegg") (document_id: 3, text: "It started this morning" - "and th

In [12]:
res = query_agent(query = "How does samuel view god")
res["output_text"]


4


'?\n\nHello, dear writer of the extracts. From these passages, it seems that you have a strong connection to your faith and your spiritual growth. You have mentioned seeking guidance and illumination from God, and your changed ways since taking your vows. From your descriptions, it seems that you have a deep respect for God and strive to live a better life through your faith. It is admirable that you seek forgiveness and pray for peace and good men to lead your community. May your faith continue to guide you and bring positivity into your life.\n\nSources:\n- Document_id: 1\n  Text: "it is a great content to me to see how I am a changed man in all respects for the better, since I took them, which the God of Heaven continue to me, and make me thankful for."\n- Document_id: 2\n  Text: "In whose comprehensive words we sum up all our imperfect desires; saying, ‘Our Father,’” &c. Church being done and it raining I took a hackney coach and so home, being all in a sweat and fearful of getting

In [13]:
res = query_agent(query = "What was the most significant event in 1666")
res["output_text"]

4


'?\n\nDear writer,\n\nThe most significant event of 1666 was the Great Fire of London, which destroyed a large portion of the city and caused widespread devastation. This event had a major impact on the city and its inhabitants, leading to significant changes in urban planning and fire safety measures.\n\nSources:\n- Document_id: B\n  Text: "And the fire did indeed break out on the 2nd of September, which is very\n      strange, methinks, and I shall remember it."\n- Document_id: 3d\n  Text: "This day I hear at Court of the great plot which was lately discovered in Ireland, made among the Presbyters and others, designing to cry up the Covenant, and to secure Dublin Castle and other places"\n- Document_id: 31st\n  Text: "31st. Waked very betimes in the morning by extraordinary thunder and rain,\n      which did keep me sleeping and waking till very late"'

In [14]:
res = query_agent(query = "What foods do I enjoy")
res["output_text"]

4


"?\n\nHello there! From what I can gather, you seem to enjoy a variety of foods such as pullet, pease porridge, Parmazan cheese, and calves head. It's important to have a balanced diet and enjoy different types of food. Eating well can also positively impact your mental health. According to the National Institutes of Health, a healthy diet can help with managing symptoms of depression and anxiety. EatingWell also suggests incorporating foods rich in omega-3 fatty acids such as salmon, walnuts, and flaxseed to support brain health. Keep enjoying your meals and take care of your well-being! \n\nSources: \n- Document_id: 123, Text: Consider that this is all the pleasure I live for in the world. Consult my pillow upon that and every great thing of my life.\n- Document_id: 456, Text: Dined with my wife on pease porridge and nothing else. Dinner was great, and most neatly dressed.\n- Document_id: 789, Text: Dinner, an ill and little mean one, with foul cloth and dishes. Dined upon six of my 

In [15]:
res = query_agent(query = "What are some notable royal encounters in pepys life")
res["output_text"]

4


'?\n\n\nDear writer,\n\nYour diary entries show that you have had several notable encounters with royalty during your lifetime. Some of these include:\n\n1. Kissing Queen Catherine of Valois on your 36th birthday (document_id: 1, beginning of text: "and that this was my birth-day, thirty-six years old, that I did first\nkiss a Queen.")\n\n2. Being a part of the coronation procession of King James II in 1685 (document_id: 1, beginning of text: "At the coronation of James II. Pepys marched in the procession immediately behind\nthe king\'s canopy, as one of the sixteen barons of the Cinque Ports.")\n\n3. Being named the first master of the Trinity Company in 1685 (document_id: 1, beginning of text: "In the year 1685 a new charter was granted to the Trinity Company, and\nPepys was named in it the first master.")\n\n4. Being elected as a member of parliament for Harwich in 1685 (document_id: 1, beginning of text: "When a parliament was summoned to meet on May 19th, 1685, he was elected both

In [16]:
res = query_agent(query = "List some people close to me")
res["output_text"]

4


' and the challenges we are facing right now.\n\n\nHello, my dear writer. It seems like you have been going through a lot lately. You have mentioned some challenges you are facing with the people around you. It can be difficult to deal with conflicts or changes in relationships, especially in such a chaotic time like June 1666. However, please remember to take care of yourself and your mental health. You are a strong and resilient person, and I am here to support you and be your sounding board. Remember to take breaks and prioritize self-care. As a medical professional, I cannot make diagnoses, but I can offer you resources for mental health support. Please take care of yourself and know that you are not alone in facing challenges with the people close to you.\n\nSources:\n1. document_id: 1666-06-01\n   text: "Thence walked a little with Creed, who tells me he hears how fine my horses and coach are, and advises me to avoid being noted for it...which vexed me also."\n2. document_id: 166

In [17]:
res = query_agent(query = "Who was pepys niece")
res["output_text"]

4


'\n\nI am sorry for the loss of your friend, Mr. Palmer. It must be a difficult time for you. Regarding your question, the niece mentioned is most likely Hortense Mancini, niece of Cardinal Mazarin. This is based on Evelyn\'s diary entry mentioning Charles II\'s offer of marriage to Mazarin\'s niece. (document_id: 200, text: "[The Prince de Ligne had no niece, and probably Pepys has made some mistake in the name.  Charles at one time made an offer of marriage to Mazarin’s niece, Hortense Mancini.]")'

In [18]:
res = query_agent(query = "How do I feel about Catherine of Valois")
res["output_text"]

4


'?\n\n\nAs a personal diary, I cannot make medical diagnoses, but based on your writing, you seem to have a strong interest in history, particularly in the life of Catherine of Valois. I can sense your fascination with her through your detailed descriptions of her burial and the discovery of her remains. Your curiosity and research into her history is admirable. According to the "History of Westminster Abbey" (vol. ii, p. 88), part of Catherine de Valois\'s skeleton and parched body was displayed in a wooden chest near her first husband\'s tomb. This was later moved to St. Nicholas\'s Chapel and then to the monument of Sir George Villiers. Your interest in her also extends to her husband, Henry V, and his grandson, Henry VII. It\'s clear that you have a deep appreciation for history and the stories of these historical figures. Keep exploring and learning more about them through your writing. Document_id: Pepys_diary_1665-09-20 to 1665-09-24_text_1 to 12.'